In [44]:
import pandas as pd
# Read dataframe
malConll = pd.read_csv('malExpanded.conllu')
# del malConll['Unnamed: 0']
malConll['word'] = malConll['word'].astype(str)
malConll['lemma'] = malConll['lemma'].astype(str)
malConll['DEPS'] = malConll['DEPS'].astype(str)
type(malConll['word'].iloc[0])
malConll.columns

/home/ashim/indicmt/env_mt/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3044: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Index(['Unnamed: 0', 'sentID', 'wordPosition', 'word', 'lemma', 'UPOS', 'XPOS',
       'FEATS', 'HEAD', 'DEPREL', 'DEPS', 'MISC', 'category', 'gender',
       'number', 'person', 'case', 'vib', 'tam'],
      dtype='object')

In [83]:
sentID_group_obj = malConll.groupby('sentID')
sent_ids = list(sentID_group_obj.groups.keys())
g1 = sentID_group_obj.groups['malayalam/final data - validated/03-06-2017/agriculture/Depn/cross_breeding_of_coconut_trees.depn.ssf:1']

In [76]:
malConll[malConll.index.isin(g1)].columns

Index(['Unnamed: 0', 'sentID', 'wordPosition', 'word', 'lemma', 'UPOS', 'XPOS',
       'FEATS', 'HEAD', 'DEPREL', 'DEPS', 'MISC', 'category', 'gender',
       'number', 'person', 'case', 'vib', 'tam'],
      dtype='object')

In [108]:
import numpy as np
import random
pd.options.display.max_colwidth = 100
vibs_dep_group = malConll.groupby(['vib', 'DEPS'])
for key, item in vibs_dep_group:
    print(key)
    print(vibs_dep_group.get_group(key)['lemma'])
    break
# type(vibs_dep_group.groups.keys())
# pd.unique(vibs_dep_group.get_group(("ഉ്", 'k4'))['lemma'])
type(vibs_dep_group.get_group(("ഉ്", 'k4')))
vibs_dep_group_df = vibs_dep_group.get_group(("ഉ്", 'k4'))
type(vibs_dep_group_df[~vibs_dep_group_df['lemma'].isin(['ഗംഗാസ്നാന'])])
df_test = vibs_dep_group_df[~vibs_dep_group_df['lemma'].isin(['ഗംഗാസ്നാന'])]
df_test.ix[np.random.choice(df_test.index, 1)]['Unnamed: 0']

("''", 'ROOT')
101924    ''
Name: lemma, dtype: object


/home/ashim/indicmt/env_mt/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  from ipykernel import kernelapp as app


43961    43961
Name: Unnamed: 0, dtype: int64

In [143]:
# Group by Object on vib and DEPS
vibs_dep_group = malConll.groupby(['vib', 'DEPS'])
vibs_dep_group_keys = vibs_dep_group.groups.keys()

word_index_to_df = {}
index_none = set()

print("Finding possible replacements for words...")
# Get Nonce and corresponding incorrect sentences
def get_possible_replacements(row):

    if pd.isnull(row['vib']) or pd.isnull(row['DEPS']):
        return None
    
    key_to_match = (row['vib'], row['DEPS'])
    # Can Not find nonce if for a word there's no other lemma with same vib, DEPS
    if key_to_match not in vibs_dep_group_keys:
        return None            
    # Dataframe
    same_vib_deps_group = vibs_dep_group.get_group(key_to_match)
    # Dataframe (Same vib and DEPS - Different lemma)
    possible_replacements = same_vib_deps_group[~same_vib_deps_group['lemma'].isin([row['lemma']])]
    
    return possible_replacements

for i, sid in enumerate(sent_ids):
    
    if i %1000 == 0:
        print("Sentence IDs Done {0}/{1}".format(i, len(sent_ids)))
    
    # Indices to words for each sentence (type: Int64Index([]))
    sentence_index = sentID_group_obj.groups[sid]
    sentence_df = malConll[malConll.index.isin(sentence_index)]
    
    for index, row in sentence_df.iterrows():
        ret_value = get_possible_replacements(row)
        if ret_value is None:
            index_none.add(index)
        elif ret_value.empty:
            index_none.add(index)
            ret_value = None
        word_index_to_df[index] = ret_value

# row['DEPS'].startswith('k')
print(len(index_none))

print("Finding sentences that don't have any possible nonce sentence...")
skip_sent_ids = set()
for i, sid in enumerate(sent_ids):
    
    if i %1000 == 0:
        print("Sentence IDs Done {0}/{1}".format(i, len(sent_ids)))
    
    # Indices to words for each sentence (type: Int64Index([]))
    sentence_index = sentID_group_obj.groups[sid]
    sentence_df = malConll[malConll.index.isin(sentence_index)]
    
    for index, row in sentence_df.iterrows():
        if index in index_none:
            skip_sent_ids.add(sid)

print("Total Sentence IDs : {0}".format(len(sent_ids)))
print("To be skipped : {0}".format(len(skip_sent_ids)))

Finding possible replacements for words...
Sentence IDs Done 0/13370
Sentence IDs Done 1000/13370
Sentence IDs Done 2000/13370
Sentence IDs Done 3000/13370
Sentence IDs Done 4000/13370
Sentence IDs Done 5000/13370
Sentence IDs Done 6000/13370
Sentence IDs Done 7000/13370
Sentence IDs Done 8000/13370
Sentence IDs Done 9000/13370
Sentence IDs Done 10000/13370
Sentence IDs Done 11000/13370
Sentence IDs Done 12000/13370
Sentence IDs Done 13000/13370
61034
Finding sentences that don't have any possible nonce sentence...
Sentence IDs Done 0/13370
Sentence IDs Done 1000/13370
Sentence IDs Done 2000/13370
Sentence IDs Done 3000/13370
Sentence IDs Done 4000/13370
Sentence IDs Done 5000/13370
Sentence IDs Done 6000/13370
Sentence IDs Done 7000/13370
Sentence IDs Done 8000/13370
Sentence IDs Done 9000/13370
Sentence IDs Done 10000/13370
Sentence IDs Done 11000/13370
Sentence IDs Done 12000/13370
Sentence IDs Done 13000/13370
Total Sentence IDs : 13370
To be skipped : 13367


In [152]:
# Group by Object on vib and DEPS
deps_lemma_group = malConll.groupby(['DEPS', 'lemma'])
deps_lemma_group_keys = deps_lemma_group.groups.keys()

word_index_to_df_nonce = word_index_to_df
word_index_to_df_incorrect = {}
index_incorrect_none = set()


def get_possible_incorrects(row):

    if pd.isnull(row['vib']) or pd.isnull(row['DEPS']):
        return None
    
    if not row['DEPS'].startswith('k'):
        return None
    
    key_to_match = (row['DEPS'], row['lemma'])
    # Can Not find nonce if for a word there's no other lemma with same vib, DEPS
    if key_to_match not in deps_lemma_group_keys:
        return None
    
    same_deps_lemma_group = deps_lemma_group.get_group(key_to_match)
    possible_incorrects = same_deps_lemma_group[~same_deps_lemma_group['vib'].isin([row['vib']])]
    return possible_incorrects

for i, sid in enumerate(sent_ids):
    
    if i %1000 == 0:
        print("Sentence IDs Done {0}/{1}".format(i, len(sent_ids)))
    
    # Indices to words for each sentence (type: Int64Index([]))
    sentence_index = sentID_group_obj.groups[sid]
    sentence_df = malConll[malConll.index.isin(sentence_index)]
    
    for index, row in sentence_df.iterrows():
        ret_value = get_possible_incorrects(row)
        if ret_value is None:
            index_incorrect_none.add(index)
        elif ret_value.empty:
            index_incorrect_none.add(index)
            ret_value = None
        word_index_to_df_incorrect[index] = ret_value
    
print(len(index_incorrect_none))

Sentence IDs Done 0/13370
Sentence IDs Done 1000/13370
Sentence IDs Done 2000/13370
Sentence IDs Done 3000/13370
Sentence IDs Done 4000/13370
Sentence IDs Done 5000/13370
Sentence IDs Done 6000/13370
Sentence IDs Done 7000/13370
Sentence IDs Done 8000/13370
Sentence IDs Done 9000/13370
Sentence IDs Done 10000/13370
Sentence IDs Done 11000/13370
Sentence IDs Done 12000/13370
Sentence IDs Done 13000/13370
121465


In [151]:
print(len(word_index_to_df))
required_deps = ['k1', 'k2', 'k3', 'k4', 'k5', 'k6', 'k7']

sentences_dict = {}

traversed = {}


def get_new_nonce_sentence(sid):
    if sid not in traversed:
        traversed[sid] = set()
#     if sid in skip_sent_ids:
#         continue
    # Indices to words for each sentence (type: Int64Index([]))
    sentence_index = sentID_group_obj.groups[sid]
    sentence_df = malConll[malConll.index.isin(sentence_index)]
    
    nonce_sentence = []
    
    for index, row in sentence_df.iterrows():
        
        possible_replacements = word_index_to_df[index]
        if possible_replacements.empty:
            return get_new_nonce_sentence(sid)
        random_idx = np.random.choice(possible_replacements.index, 1).tolist()[0]
        nonce_sentence.append(random_idx)
    
    nonce_sent_index_str = ' '.join([str(ns) for ns in nonce_sentence])
    if nonce_sent_index_str in traversed[sid]:
        return get_new_nonce_sentence(sid)
    else:
        return nonce_sentence

def get_incorrect_sentence(nonce_idx):
    nonce_sentence_df = malConll[malConll.index.isin(nonce_idx)]   
    
for sid in sent_ids:
    
    if sid not in traversed:
        traversed[sid] = {}
    
    if sid in skip_sent_ids:
        continue
    nonce_idx = get_new_nonce_sentence(sid)
    print("{0} : {1}".format(sid, nonce_idx))
    get_incorrect_sentence(nonce_idx)
    break

132834
malayalam/final data - validated/11-05-2017/tourism (parallel)/Depn/Mal_tourism 1.depn.ssf:125 : [4715, 27569, 111714, 88097]
        Unnamed: 0  \
4715          4715   
27569        27569   
88097        88097   
111714      111714   

                                                                                                sentID  \
4715           malayalam/full data as on 25-01-2017/not validated/health/depn/MalText83_19.depn.ssf:91   
27569      malayalam/full data as on 25-01-2017/not validated/agriculture/depn/MalText74_7.depn.ssf:28   
88097                   malayalam/final data - validated/15-07-2017/tourism/Depn/MalText42.depn.ssf:98   
111714  malayalam/icon 2016/folder 1 - validated/13dec2016 - batch1/Dependency/MalText53_7.depn.ssf:31   

        wordPosition         word  lemma     UPOS XPOS  \
4715               4         അളവു   അളവ്     N_NN    _   
27569              2  കേരളത്തില്‍   കേരള   N__NNP    _   
88097              3        എത്തി  എത്ത്  V_VM_VF  